In [56]:
import os,glob
import pandas as pd
import nibabel as nib
DATA_PATH="datasets/adData_Huashan"
TAR_PATH="datasets/adData_Huashan_prep"
CSV_PATH="datasets/adData_Huashan/Diagnosis Information.csv"
%cd AD_GAN

c:\Users\CH2\Projects\AD_GAN


In [49]:
df=pd.read_csv(CSV_PATH,dtype=str,keep_default_na=False)
# df.set_index("PID")
pdict={value:key for key,value in df[df["diagonsis"]!=""]["PID"].to_dict().items()}
pdict

{'ZS13339215': 0,
 'ZS12344318': 1,
 'ZS20108556': 2,
 'ZS20270289': 3,
 'ZS20270288': 4,
 'ZS09005057': 5,
 'ZS09176714': 6,
 'ZS09069673': 7,
 'ZS08002918': 8,
 'ZS20276801': 9,
 'ZS20276793': 10,
 'ZS20276785': 11,
 'ZS11344100': 12,
 'ZS20276854': 13,
 'ZS20276850': 14,
 'ZS20276845': 15,
 'ZS17142399': 16,
 'ZS17281196': 17,
 'ZS16268152': 18,
 'ZS10312675': 19,
 'ZS13229156': 20,
 'ZS15049778': 21,
 'ZS11070647': 22,
 'ZS09037906': 23,
 'ZS14049399': 24,
 'ZS20283449': 25,
 'ZS20283451': 26,
 'ZS20283452': 27,
 'ZS20283454': 28,
 'ZS15149810': 29,
 'ZS20283719': 30,
 'ZS20283720': 31,
 'ZS09067464': 32,
 'ZS09128145': 33,
 'ZS09004601': 34,
 'ZS13004626': 35,
 'ZS09024393': 36,
 'ZS21007410': 37,
 'ZS16043037': 38,
 'ZS15168778': 39,
 'ZS19075135': 40,
 'ZS12151383': 41,
 'ZS18247502': 42,
 'ZS18020720': 43,
 'ZS16319770': 44,
 'ZS10015525': 45,
 'ZS09150731': 46,
 'ZS14103875': 47,
 'ZS14249195': 48,
 'ZS13090589': 50,
 'ZS13069533': 51,
 'ZS09230831': 52,
 'ZS21056384': 53,
 'Z

In [ ]:
patient_dict={}
for dirpath, dirnames, filenames in os.walk("."):
    pid=os.path.basename(dirpath).split("_")
    for poss_h in pid:
        if poss_h in pdict:
            df_id=pdict[poss_h]
            diag=df.iloc()[df_id]["diagonsis"]
            if diag not in patient_dict:patient_dict[diag]=[]
            patient_dict[diag].append([poss_h,dirpath])


In [ ]:
import subprocess

def show(s,output=None):
    print(s,flush=True)
    # print(s,file=sys.stderr,flush=True)
    if output is not None:
        output.append(s+'\n')

def run_sh(cmd,name="unknown",base_dir="tmp",pname="unknown",outputs=[]):
    # tcmd=
    outputs.append(cmd)
    ret=subprocess.run(f"cd {base_dir} && {cmd} 2>&1",shell=True,stdout=subprocess.PIPE,encoding="utf")
    if ret.stdout is not None:
        outputs.append(ret.stdout+'\n')
    if ret.returncode!=0:
        show(f"{pname}.{name}: {cmd.split()[0]} Failed!",outputs)
        raise Exception(f"{name} Error!")
    show(f"{pname}.{name}: {cmd.split()[0]} Successfully!",outputs)


In [ ]:
def load_t1_img(srcpath,tgpath):
    os.makedirs(tgpath,exist_ok=True)
    sh=lambda cmd,name="unknown",base_dir=".":run_sh(cmd,name=name,base_dir=base_dir,pname=srcpath,outputs=[])
    srcpath=srcpath+"/Image/t1_gre_fsp_3d_sag_0.8mm_[0-9]*"
    srcpath=sorted(glob.glob(srcpath))[0]
    sh(f"cp -t {tgpath} {srcpath}/*",name="cp",base_dir=srcpath)
    sh(f"dcm2niix -b y -z y -x n -t n -m n -f t1 -o . -s n -v n .",name="dcm2nii",base_dir=tgpath)
    img=nib.load("t1.nii.gz")
    img_arr=img.get_fdata()
    return img_arr

In [ ]:
os.makedirs(TAR_PATH,exist_ok=True)
NC_PATH,MCI_PATH=TAR_PATH+"/NC",TAR_PATH+"/MCI"
os.makedirs(NC_PATH,exist_ok=True)
os.makedirs(MCI_PATH,exist_ok=True)
data=[]
for pid,dirpath in patient_dict["NC"]:
    data.append([load_t1_img(dirpath,NC_PATH+"/"+pid),0])
for pid,dirpath in patient_dict["MCI"]:
    data.append([load_t1_img(dirpath,MCI_PATH+"/"+pid),1])

In [54]:
df.iloc()[49]["diagonsis"]

''